In [134]:
import pandas as pd

# Q1. Downloading the data

In [135]:
jan = pd.read_parquet('fhv_tripdata_2021-01.parquet')
feb = pd.read_parquet('fhv_tripdata_2021-02.parquet')

## Read the data for January. How many records are there?

In [136]:
jan.shape

(1154112, 7)

# Q2. Computing duration
## Now let's compute the duration variable. It should contain the duration of a ride in minutes.
## What's the average trip duration in January?

In [137]:
jan.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
dtype: object

In [138]:
jan['duration'] = (jan['dropOff_datetime'] - jan['pickup_datetime'])/pd.Timedelta(minutes=1)

In [139]:
round(jan['duration'].mean(),3)

19.167

In [140]:
jan = jan[(jan.duration >= 1) & (jan.duration <= 60)]

# Q3. Missing values
## The features we'll use for our model are the pickup and dropoff location IDs.

## But they have a lot of missing values there. Let's replace them with "-1".

## What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

In [141]:
jan.isna().sum()

dispatching_base_num            0
pickup_datetime                 0
dropOff_datetime                0
PUlocationID               927008
DOlocationID               147907
SR_Flag                   1109826
Affiliated_base_number        773
duration                        0
dtype: int64

In [142]:
jan['PUlocationID'].fillna(-1, inplace = True)
jan['DOlocationID'].fillna(-1, inplace = True)
jan['PUlocationID'].value_counts()

-1.0      927008
 221.0      8330
 206.0      6797
 129.0      5379
 115.0      4082
           ...  
 111.0         5
 27.0          4
 34.0          3
 2.0           2
 110.0         1
Name: PUlocationID, Length: 262, dtype: int64

In [143]:
jan['PUlocationID'].value_counts()[-1]/len(jan)

0.8352732770722617

# Q4. One-hot encoding
## Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

## Turn the dataframe into a list of dictionaries
## Fit a dictionary vectorizer
## Get a feature matrix from it
## What's the dimensionality of this matrix? (The number of columns).

In [144]:
from sklearn.feature_extraction import DictVectorizer

In [145]:
df = jan.loc[:,['PUlocationID','DOlocationID']]

In [146]:
# turn X into dict
df = df.astype(str)
X_dict = df.to_dict(orient='records') # turn each row as key-value pairs
# show X_dict


In [147]:
# DictVectorizer
from sklearn.feature_extraction import DictVectorizer
# instantiate a Dictvectorizer object for X
dv_X = DictVectorizer() 


In [148]:
# apply dv_X on X_dict
X_encoded = dv_X.fit_transform(X_dict)
# show X_encoded
X_encoded

<1109826x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

# Q5. Training a model
## Now let's use the feature matrix from the previous step to train a model.

## Train a plain linear regression model with default parameters
## Calculate the RMSE of the model on the training data
## What's the RMSE on train?

In [149]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

y = jan['duration'].values
lr = LinearRegression()
lr.fit(X_encoded,y)
print(mse(lr.predict(X_encoded),y))

110.84971459076822


In [150]:
print(mse(y,lr.predict(X_encoded))**0.5)

10.528519107204405


# Q6. Evaluating the model
## Now let's apply this model to the validation dataset (Feb 2021).

## What's the RMSE on validation?

In [151]:
def prepare_df(df,dv_X):

    df['PUlocationID'].fillna(-1, inplace = True)
    df['DOlocationID'].fillna(-1, inplace = True)
    df['duration'] = (df['dropOff_datetime'] - df['pickup_datetime'])/pd.Timedelta(minutes=1)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    y = df['duration'].values
    df = df.loc[:,['PUlocationID','DOlocationID']]
    df = df.astype(str)
    X_dict = df.to_dict(orient='records') # turn each row as key-value pairs
    from sklearn.feature_extraction import DictVectorizer



    X_encoded = dv_X.transform(X_dict)


    return X_encoded, y

In [152]:
X_test, y_test = prepare_df(feb,dv_X)

In [153]:
X_test

<990113x525 sparse matrix of type '<class 'numpy.float64'>'
	with 1980223 stored elements in Compressed Sparse Row format>

In [154]:
mse(y_test,lr.predict(X_test),squared=False)

11.014283137481941